In [ ]:
xgb_model_for_new_feature

In [ ]:
新特征的xgb

In [ ]:
数据集B

In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn import metrics

E:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
线下训练部分：
1-16训练，8-24验证
线上提交部分：
1-16和8-24都训练，然后15-30用两个model predict结果

In [ ]:
读取数据集

In [ ]:
先只读精简的

In [2]:
table1_16 = pd.read_csv('dataset_B_for_model_method/final_dataset/dataset_1_16.csv')
table8_23 = pd.read_csv('dataset_B_for_model_method/final_dataset/dataset_8_23.csv')
table15_30 = pd.read_csv('dataset_B_for_model_method/final_dataset/dataset_15_30.csv')

In [3]:
table1_16.head()

,user_id,launch_day_near_pred,launch_day_sum,launch_day_sum_average,launch_day_gap_max,launch_day_gap_ain,launch_day_gap_average,launch_day_gap_std,launch_day_gap_skew,launch_day_gap_kurtosis,...,act_author_count_average,act_author_count_max,act_author_count_max_ratio,act_author_count_std,act_author_count_skew,act_author_count_kurtosis,y,register_day,register_type,device_type
0,8,3.0,4,0.80,2.0,1.0,1.333333,0.471405,0.707107,-1.5,...,1.600000,7.0,0.072917,1.280625,2.586390,6.403331,1.0,9,0,34
1,73,11.0,3,0.75,2.0,1.0,1.500000,0.500000,0.000000,-2.0,...,2.266667,10.0,0.294118,2.619584,2.044814,2.826957,0.0,2,1,947
2,129,4.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,...,1.000000,1.0,1.000000,0.000000,0.000000,-3.000000,1.0,12,1,43
3,228,3.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,...,22.000000,22.0,1.000000,0.000000,0.000000,-3.000000,1.0,13,2,13
4,351,13.0,1,1.00,-1.0,-1.0,-1.000000,-1.000000,-99.000000,-99.0,...,3.744681,31.0,0.176136,5.777683,3.471943,12.324511,0.0,3,1,2042


In [4]:
table1_16.shape

(23923, 96)

In [ ]:
先看看有哪些参数基本上取值非常的单一（看std）

In [6]:
std_list = []
for i in range(1,table1_16.shape[1]):
    temp = table1_16.iloc[:,i]
    std_list.append(np.std(temp))

In [7]:
std_list[0],np.std(table1_16['launch_day_near_pred'])

(4.0818123458765205, 4.0818123458765205)

In [8]:
columns = list(table1_16.columns)[1:]
std_pd = pd.Series(data = std_list,index = columns)
std_pd.sort_values(ascending = True,inplace = True)

In [ ]:
可以决定删一些

In [23]:
drop_list = list(std_pd.index[:5])

In [24]:
drop_list

['action_type_4',
 'action_type_5',
 'action_type_3',
 'action_type_2',
 'activity_page_4']

In [18]:
std_pd.head(10)  

action_type_4             0.000304
action_type_5             0.002988
action_type_3             0.026078
action_type_2             0.039876
activity_page_4           0.066387
action_type_1             0.098203
action_type_0             0.109616
activity_page_1           0.193782
launch_day_sum_average    0.216042
act_day_sum_average       0.221143
dtype: float64

In [5]:
table1_16_y = table1_16.y
table1_16_x = table1_16.drop(['user_id','y'],axis = 1)
table8_23_y = table8_23.y
table8_23_x = table8_23.drop(['user_id','y'],axis = 1)
table15_30_x = table15_30.drop(['user_id'],axis = 1)

In [6]:
table15_30_pred = table15_30[['user_id']]

In [7]:
table15_30.shape,table15_30_pred.shape

((51480, 95), (51480, 1))

In [ ]:
筛选特征

In [ ]:
1 用drop_list筛选掉部分得分最低的特征：从特征重要性角度（下边的drop_list）

In [22]:
dataset1 = xgb.DMatrix(table1_16_x.drop(drop_list,axis = 1),label = table1_16_y)
dataset2 = xgb.DMatrix(table8_23_x.drop(drop_list,axis = 1),label = table8_23_y)
dataset3 = xgb.DMatrix(table15_30_x.drop(drop_list,axis = 1))

In [ ]:
2 从特征std的角度，删除一些值很稳定没有什么变化的特征（上边的drop_list）

In [29]:
dataset1 = xgb.DMatrix(table1_16_x.drop(drop_list,axis = 1),label = table1_16_y)
dataset2 = xgb.DMatrix(table8_23_x.drop(drop_list,axis = 1),label = table8_23_y)
dataset3 = xgb.DMatrix(table15_30_x.drop(drop_list,axis = 1))

In [ ]:
原始，不筛选的特征构造

In [8]:
dataset1 = xgb.DMatrix(table1_16_x,label = table1_16_y)
dataset2 = xgb.DMatrix(table8_23_x,label = table8_23_y)
dataset3 = xgb.DMatrix(table15_30_x)

In [ ]:
# 保留参数
params={
    'booster':'gbtree',
    'objective':'binary:logistic'
    # 'eval_metric': 'auc', # 评价标准是f1，因此需要自己写feval
    'tree_method':'exact', # 对于小数据集，xgb使用原版（贪心算法），不需要使用近似算法
    'gamma':0.1,
    'max_depth':5,
    'lambda':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':1.1, 
    'colsample_bylevel':0.7,
    'eta': 0.01, 
    'seed':1000,
    }

In [ ]:
计算f1值的def，用于train中

In [70]:
def evalerror(preds,dtrain):
    labels = dtrain.get_label()
    # print(labels)
    # print(preds)
    for i in range(len(preds)):
        if preds[i] > 0.4:
            preds[i] = 1
        else:
            preds[i] = 0
    
    f1_score = metrics.f1_score(labels, preds, average='weighted')  
    return 'f1_score',f1_score

In [ ]:
走一遍流程的def：从train到pred

In [9]:
def pipeline(random_seed,
             gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight,colsample_bylevel,
             dataset1,dataset2,num_boost_round_value,threshold_value,early_stopping_rounds_value):
    
    params={
    'booster':'gbtree',
    'objective':'binary:logistic',
    # 'eval_metric': 'auc', # 评价标准是f1，因此需要自己写feval
    'tree_method':'exact', # 对于小数据集，xgb使用原版（贪心算法），不需要使用近似算法
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd, # 10
    'subsample':subsample, # 0.7
    'colsample_bytree':colsample_bytree, # 0.7
    'min_child_weight':min_child_weight,  # 1.1
    'colsample_bylevel':colsample_bylevel, # 0.7
    'eta': 0.01, 
    'seed':1000,
    'eval_metric': 'logloss' # 试试loss吧
    }
    
    # model1
    watchlist = [(dataset1,'train')]
    # model_1 = xgb.train(params,dataset1,num_boost_round = num_boost_round_value,evals = watchlist,feval = evalerror)
    model_1 = xgb.train(params,dataset1,num_boost_round = num_boost_round_value,evals = watchlist,
                        early_stopping_rounds = early_stopping_rounds_value)
    

    feature_score_1 = model_1.get_fscore()
    feature_score_1 = sorted(feature_score_1.items(), key=lambda x:x[1],reverse=True)

    preds_1 = model_1.predict(dataset2)

    preds1_01 = cal_preds(preds_1,threshold_value)
    print('model1的：1的个数和全部的个数')
    print(np.sum(preds1_01),len(preds1_01))

    # 上边这个f1计算方式不太对呢
    # f1_score_1 = metrics.f1_score(dataset2.get_label(), preds1_01, average='weighted')
    accu_1 = metrics.accuracy_score(dataset2.get_label(), preds1_01)
    recall_1 = metrics.recall_score(dataset2.get_label(), preds1_01)
    f1_score_1 = (2 * accu_1 * recall_1 ) / (accu_1 + recall_1) 
    print('指标结果:f1，准确率，召回率')
    print(f1_score_1,accu_1,recall_1)
    
    # 使用8-23的特征，去predict 1-16的结果，不太合理。
    '''
    # model2
    watchlist = [(dataset2,'train')]
    model_2 = xgb.train(params,dataset2,num_boost_round = num_boost_round_value,evals = watchlist,feval = evalerror)

    feature_score_2 = model_2.get_fscore()
    feature_score_2 = sorted(feature_score_2.items(), key=lambda x:x[1],reverse=True)

    preds_2 = model_2.predict(dataset1)

    preds2_01 = cal_preds(preds_2,threshold_value)
    print('model2的：1的个数和全部的个数')
    print(np.sum(preds2_01),len(preds2_01))

    f1_score_2 = metrics.f1_score(dataset1.get_label(), preds2_01, average='weighted')
    '''
    
    # return model_1,model_2,feature_score_1,feature_score_2,f1_score_1,f1_score_2

    return model_1,feature_score_1,f1_score_1,preds_1

In [ ]:
pipeline内部使用的二分数值def

In [10]:
def cal_preds(preds,val):    
    preds01 = []
    for i in range(len(preds)):
        if preds[i] > val:
            preds01.append(1)
        else:
            preds01.append(0)
    return preds01

In [ ]:
(只有pipeline确定参数后才会执行这个def)
先使用dataset2训练model2
然后用model1和model2都pred testset

In [11]:
def model_train_for_testset(random_seed,
             gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight,colsample_bylevel,
             model_1,dataset2,dataset3,num_boost_round_value,threshold_value,early_stopping_rounds_value):
    
    params={
    'booster':'gbtree',
    'objective':'binary:logistic',
    # 'eval_metric': 'auc', # 评价标准是f1，因此需要自己写feval
    'tree_method':'exact', # 对于小数据集，xgb使用原版（贪心算法），不需要使用近似算法
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd, # 10
    'subsample':subsample, # 0.7
    'colsample_bytree':colsample_bytree, # 0.7
    'min_child_weight':min_child_weight,  # 1.1
    'colsample_bylevel':colsample_bylevel, # 0.7
    'eta': 0.01, 
    'seed':1000,
    'eval_metric':'logloss'
    }
    
    # model2
    watchlist = [(dataset2,'train')]
    # model_2 = xgb.train(params,dataset2,num_boost_round = num_boost_round_value,evals = watchlist,feval = evalerror)
    model_2 = xgb.train(params,dataset2,num_boost_round = num_boost_round_value,evals = watchlist,
                       early_stopping_rounds = early_stopping_rounds_value)

    feature_score_2 = model_2.get_fscore()
    feature_score_2 = sorted(feature_score_2.items(), key=lambda x:x[1],reverse=True)

    # preds_2 = model_2.predict(dataset1) # 不能用将来的去pred过去的，这个model的作用是predict dataset3的\
    preds_2 = model_2.predict(dataset3)

    preds2_01 = cal_preds(preds_2,threshold_value)
    print('model2的：1的个数和全部的个数')
    print(np.sum(preds2_01),len(preds2_01))

    # f1_score_2 = metrics.f1_score(dataset1.get_label(), preds2_01, average='weighted') 显然也是没法计算f1值的
    
    # model1
    preds_1 = model_1.predict(dataset3)

    preds1_01 = cal_preds(preds_1,threshold_value)
    print('model1的：1的个数和全部的个数')
    print(np.sum(preds1_01),len(preds1_01))   

    return model_2,feature_score_2,preds2_01,preds1_01,preds_2,preds_1

## 调参

### 基本baseline

In [ ]:
先给出dataset2的真实的活跃用户个数
——17996（18000）

In [11]:
table8_23_y.value_counts()

0.0    19339
1.0    17996
Name: y, dtype: int64

In [12]:
table1_16_y.value_counts()

1.0    12430
0.0    11493
Name: y, dtype: int64

In [ ]:
首先明确：
threshold_value是model train完之后的参数，调整0和1的分界线用的。
——实际上因此它不影响model的【效果】
——但是，是会影响最终的F1值的！因此最好拿0.4来处理

In [12]:
model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = 0.1,max_depth = 5,lambd = 0,subsample = 0.95,colsample_bytree = 0.95,
    min_child_weight = 1,colsample_bylevel = 0.95,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 600,threshold_value = 0.30,
early_stopping_rounds_value = 100)

[0]	train-logloss:0.688836
Will train until train-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.684602
[2]	train-logloss:0.68045
[3]	train-logloss:0.676395
[4]	train-logloss:0.672397
[5]	train-logloss:0.668486
[6]	train-logloss:0.664658
[7]	train-logloss:0.660893
[8]	train-logloss:0.657196
[9]	train-logloss:0.653582
[10]	train-logloss:0.650021
[11]	train-logloss:0.64651
[12]	train-logloss:0.64306
[13]	train-logloss:0.639677
[14]	train-logloss:0.636361
[15]	train-logloss:0.633114
[16]	train-logloss:0.629929
[17]	train-logloss:0.626787
[18]	train-logloss:0.623722
[19]	train-logloss:0.620683
[20]	train-logloss:0.617711
[21]	train-logloss:0.614797
[22]	train-logloss:0.611918
[23]	train-logloss:0.609098
[24]	train-logloss:0.606321
[25]	train-logloss:0.60358
[26]	train-logloss:0.600871
[27]	train-logloss:0.59824
[28]	train-logloss:0.59563
[29]	train-logloss:0.593077
[30]	train-logloss:0.590571
[31]	train-logloss:0.588138
[32]	train-logloss:0.585725
[33]	train-logloss:0.583346
[3

[286]	train-logloss:0.419681
[287]	train-logloss:0.419561
[288]	train-logloss:0.41944
[289]	train-logloss:0.419315
[290]	train-logloss:0.419186
[291]	train-logloss:0.419056
[292]	train-logloss:0.418937
[293]	train-logloss:0.418811
[294]	train-logloss:0.418683
[295]	train-logloss:0.418563
[296]	train-logloss:0.41844
[297]	train-logloss:0.418311
[298]	train-logloss:0.418181
[299]	train-logloss:0.418065
[300]	train-logloss:0.41794
[301]	train-logloss:0.417833
[302]	train-logloss:0.41773
[303]	train-logloss:0.417616
[304]	train-logloss:0.417504
[305]	train-logloss:0.417401
[306]	train-logloss:0.417296
[307]	train-logloss:0.417196
[308]	train-logloss:0.417098
[309]	train-logloss:0.416988
[310]	train-logloss:0.416864
[311]	train-logloss:0.416749
[312]	train-logloss:0.416624
[313]	train-logloss:0.416499
[314]	train-logloss:0.41639
[315]	train-logloss:0.416279
[316]	train-logloss:0.416173
[317]	train-logloss:0.416053
[318]	train-logloss:0.415942
[319]	train-logloss:0.41582
[320]	train-logloss:

[570]	train-logloss:0.397946
[571]	train-logloss:0.397877
[572]	train-logloss:0.397802
[573]	train-logloss:0.397752
[574]	train-logloss:0.397705
[575]	train-logloss:0.397615
[576]	train-logloss:0.397579
[577]	train-logloss:0.39752
[578]	train-logloss:0.397492
[579]	train-logloss:0.397449
[580]	train-logloss:0.397424
[581]	train-logloss:0.397399
[582]	train-logloss:0.397379
[583]	train-logloss:0.397346
[584]	train-logloss:0.397295
[585]	train-logloss:0.397268
[586]	train-logloss:0.397237
[587]	train-logloss:0.397197
[588]	train-logloss:0.397134
[589]	train-logloss:0.397098
[590]	train-logloss:0.397052
[591]	train-logloss:0.396997
[592]	train-logloss:0.396947
[593]	train-logloss:0.396924
[594]	train-logloss:0.396892
[595]	train-logloss:0.396857
[596]	train-logloss:0.396817
[597]	train-logloss:0.396764
[598]	train-logloss:0.396693
[599]	train-logloss:0.396637
model1的：1的个数和全部的个数
22607 37335
指标结果:f1，准确率，召回率
0.84077887196 0.785000669613 0.905090020004


In [ ]:
看看特征选择会带来什么改变

In [15]:
temp = feature_score_1[-21:]

In [16]:
drop_list = []
for item in temp:
    drop_list.append(item[0])

In [17]:
drop_list

['act_day_gap_average',
 'act_day_gap_ain',
 'video_count_pred',
 'video_count_day_min',
 'video_day_sum_average',
 'video_day_gap_std',
 'video_day_sum',
 'act_day_gap_max_continue_flag',
 'video_count_min',
 'video_count_gap_change_small_right_continue',
 'video_day_gap_average',
 'video_day_gap_max',
 'video_day_gap_kurtosis',
 'video_day_gap_ain',
 'video_day_gap_max_right_continue',
 'video_count_kurtosis',
 'action_type_4',
 'video_day_gap_max_continue',
 'video_day_gap_skew',
 'act_count_diff_minus_right_continue',
 'video_count_max']

In [18]:
feature_score_1

[('launch_day_near_pred', 1417),
 ('device_type', 1183),
 ('act_day_near_pred', 830),
 ('act_count_sum', 692),
 ('register_type', 500),
 ('act_count_diff_plus_max', 469),
 ('activity_page_1', 469),
 ('action_type_1', 467),
 ('activity_page_3', 449),
 ('launch_day_sum', 421),
 ('act_count_pred', 416),
 ('act_author_count_max_ratio', 405),
 ('act_author_count_average', 383),
 ('action_type_0', 382),
 ('act_author_count_skew', 368),
 ('act_count_diff_sum', 339),
 ('act_count_diff_average', 324),
 ('act_author_count_std', 324),
 ('act_author_count_len', 299),
 ('act_count_diff_skew', 290),
 ('act_author_count_kurtosis', 279),
 ('act_count_average', 264),
 ('launch_day_gap_max_right_continue', 261),
 ('act_count_max', 245),
 ('launch_day_sum_average', 244),
 ('launch_day_gap_max_continue', 239),
 ('activity_page_0', 237),
 ('launch_day_gap_std', 236),
 ('act_count_std', 230),
 ('act_count_min', 229),
 ('activity_page_2', 227),
 ('launch_day_gap_average', 208),
 ('act_count_skew', 202),
 ('r

In [14]:
feature_score_2

[('launch_day_near_pred', 1539),
 ('device_type', 1159),
 ('launch_day_sum', 573),
 ('register_type', 545),
 ('register_day', 527),
 ('act_day_near_pred', 490),
 ('launch_day_gap_max_continue', 350),
 ('act_count_pred', 348),
 ('launch_day_sum_average', 324),
 ('act_count_sum', 312),
 ('activity_page_1', 296),
 ('act_count_average', 287),
 ('action_type_1', 249),
 ('activity_page_3', 230),
 ('act_author_count_len', 221),
 ('act_count_diff_average', 207),
 ('launch_day_gap_std', 203),
 ('action_type_0', 196),
 ('act_count_diff_skew', 195),
 ('act_author_count_max_ratio', 187),
 ('act_count_diff_plus_max', 181),
 ('activity_page_2', 176),
 ('act_author_count_std', 172),
 ('act_count_diff_minus_max', 164),
 ('activity_page_0', 163),
 ('action_type_3', 163),
 ('act_count_skew', 162),
 ('act_author_count_kurtosis', 158),
 ('act_count_max', 152),
 ('act_count_diff_std', 151),
 ('launch_day_gap_average', 148),
 ('action_type_2', 144),
 ('act_author_count_average', 138),
 ('launch_day_gap_max_

In [ ]:
用model1和model2都pred testset
——【注】这里用的model_1是上边train出的结果！如果不在意这次model1的pred结果，就不用让两个函数取值一样了。

In [21]:
model_2,feature_score_2,preds2_01,preds1_01_for_test,preds_2,preds_1_for_test = model_train_for_testset(
    random_seed = 1000,gamma = 0.1,max_depth = 5,lambd = 0,subsample = 0.95,colsample_bytree = 0.95,
    min_child_weight = 1,colsample_bylevel = 0.95,model_1 = model_1,dataset2 = dataset2,
    dataset3 = dataset3,num_boost_round_value = 600,threshold_value = 0.37,
early_stopping_rounds_value = 100)

[0]	train-logloss:0.68827
Will train until train-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.683492
[2]	train-logloss:0.67881
[3]	train-logloss:0.67421
[4]	train-logloss:0.6697
[5]	train-logloss:0.665288
[6]	train-logloss:0.661027
[7]	train-logloss:0.656773
[8]	train-logloss:0.652601
[9]	train-logloss:0.648513
[10]	train-logloss:0.644488
[11]	train-logloss:0.640553
[12]	train-logloss:0.63667
[13]	train-logloss:0.632871
[14]	train-logloss:0.629131
[15]	train-logloss:0.625461
[16]	train-logloss:0.621855
[17]	train-logloss:0.618331
[18]	train-logloss:0.614844
[19]	train-logloss:0.61142
[20]	train-logloss:0.60806
[21]	train-logloss:0.604759
[22]	train-logloss:0.601577
[23]	train-logloss:0.598392
[24]	train-logloss:0.595256
[25]	train-logloss:0.592185
[26]	train-logloss:0.589154
[27]	train-logloss:0.586176
[28]	train-logloss:0.583238
[29]	train-logloss:0.580364
[30]	train-logloss:0.577542
[31]	train-logloss:0.574752
[32]	train-logloss:0.572015
[33]	train-logloss:0.569311
[34]

[286]	train-logloss:0.386825
[287]	train-logloss:0.386719
[288]	train-logloss:0.386611
[289]	train-logloss:0.386507
[290]	train-logloss:0.386392
[291]	train-logloss:0.386288
[292]	train-logloss:0.386188
[293]	train-logloss:0.386079
[294]	train-logloss:0.385986
[295]	train-logloss:0.385891
[296]	train-logloss:0.385797
[297]	train-logloss:0.385704
[298]	train-logloss:0.385607
[299]	train-logloss:0.385514
[300]	train-logloss:0.385426
[301]	train-logloss:0.385335
[302]	train-logloss:0.385239
[303]	train-logloss:0.385139
[304]	train-logloss:0.385038
[305]	train-logloss:0.384941
[306]	train-logloss:0.38485
[307]	train-logloss:0.384756
[308]	train-logloss:0.384659
[309]	train-logloss:0.384571
[310]	train-logloss:0.384484
[311]	train-logloss:0.384401
[312]	train-logloss:0.384315
[313]	train-logloss:0.384236
[314]	train-logloss:0.384154
[315]	train-logloss:0.38407
[316]	train-logloss:0.383987
[317]	train-logloss:0.383909
[318]	train-logloss:0.383833
[319]	train-logloss:0.383757
[320]	train-logl

[570]	train-logloss:0.372122
[571]	train-logloss:0.372073
[572]	train-logloss:0.37202
[573]	train-logloss:0.371963
[574]	train-logloss:0.37189
[575]	train-logloss:0.371846
[576]	train-logloss:0.371828
[577]	train-logloss:0.371785
[578]	train-logloss:0.371747
[579]	train-logloss:0.371699
[580]	train-logloss:0.37164
[581]	train-logloss:0.371616
[582]	train-logloss:0.371584
[583]	train-logloss:0.371537
[584]	train-logloss:0.371495
[585]	train-logloss:0.371447
[586]	train-logloss:0.371426
[587]	train-logloss:0.37138
[588]	train-logloss:0.371335
[589]	train-logloss:0.371297
[590]	train-logloss:0.37124
[591]	train-logloss:0.371181
[592]	train-logloss:0.371121
[593]	train-logloss:0.37108
[594]	train-logloss:0.371023
[595]	train-logloss:0.370973
[596]	train-logloss:0.370941
[597]	train-logloss:0.370895
[598]	train-logloss:0.370838
[599]	train-logloss:0.370784
model2的：1的个数和全部的个数
24826 51480
model1的：1的个数和全部的个数
25251 51480


In [ ]:
反馈结果分析：真实个数 23722
直觉上，增加提交的【个数】，相当于增加可选空间，有可能提升正确人数；（召回率提升）
另一方面，准确率可能会受到影响，增加了提交人数，可能会引入错误人数（当然也有可能引入正确的），可能会降低准确率。
最好的情况——增加的提交人数，大部分都是正确的（召回率大幅度提升，准确率下降微弱）

In [ ]:
而且！【CAUTION】
从数据集的分布角度而言，24号有注册异常情况（可能有活动或者什么）。
因此不好说set2是否是应用test，
换句话说，交一个set1的 pred结果试试呢？

In [ ]:
提交基本结果
使用model2的结果。

In [ ]:
也可以试试model1的结果

In [22]:
table15_30_pred['result'] = preds1_01_for_test
result = table15_30_pred[table15_30_pred['result'] == 1]
result = result[['user_id']]
result.shape

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(25251, 1)

In [33]:
table15_30_pred['result'] = preds2_01
result = table15_30_pred[table15_30_pred['result'] == 1]
result = result[['user_id']]
result.shape

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(27354, 1)

In [48]:
result.to_csv('baseline_xgb_600_0.4.csv',index = False, header = False) # 基本调参后的结果

In [ ]:
还可以取别的阈值看看——0.4左右波动一下

In [46]:
table15_30_pred['result2'] = preds_2

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
len(table15_30_pred[table15_30_pred['result2'] > 0.41]),len(table15_30_pred[table15_30_pred['result2'] > 0.4])

(24191, 24481)

In [49]:
len(table15_30_pred[table15_30_pred['result2'] > 0.39])

24794

In [ ]:
第2种baseline（使用不同的参数）

In [74]:
model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = 0.1,max_depth = 7,lambd = 10,subsample = 0.7,colsample_bytree = 0.7,
    min_child_weight = 1.1,colsample_bylevel = 0.7,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 1000,threshold_value = 0.4)

[0]	train-f1_score:0.807784
[1]	train-f1_score:0.81305
[2]	train-f1_score:0.811765
[3]	train-f1_score:0.813195
[4]	train-f1_score:0.813192
[5]	train-f1_score:0.813742
[6]	train-f1_score:0.814002
[7]	train-f1_score:0.813498
[8]	train-f1_score:0.812874
[9]	train-f1_score:0.812655
[10]	train-f1_score:0.812492
[11]	train-f1_score:0.812531
[12]	train-f1_score:0.812991
[13]	train-f1_score:0.813495
[14]	train-f1_score:0.813831
[15]	train-f1_score:0.81408
[16]	train-f1_score:0.814332
[17]	train-f1_score:0.814082
[18]	train-f1_score:0.813285
[19]	train-f1_score:0.813952
[20]	train-f1_score:0.813825
[21]	train-f1_score:0.813615
[22]	train-f1_score:0.813533
[23]	train-f1_score:0.813284
[24]	train-f1_score:0.813828
[25]	train-f1_score:0.813992
[26]	train-f1_score:0.814537
[27]	train-f1_score:0.814831
[28]	train-f1_score:0.814414
[29]	train-f1_score:0.814582
[30]	train-f1_score:0.814665
[31]	train-f1_score:0.815251
[32]	train-f1_score:0.814916
[33]	train-f1_score:0.81479
[34]	train-f1_score:0.81495

[278]	train-f1_score:0.824375
[279]	train-f1_score:0.824542
[280]	train-f1_score:0.8245
[281]	train-f1_score:0.824501
[282]	train-f1_score:0.824584
[283]	train-f1_score:0.8245
[284]	train-f1_score:0.824626
[285]	train-f1_score:0.824751
[286]	train-f1_score:0.824793
[287]	train-f1_score:0.824584
[288]	train-f1_score:0.824626
[289]	train-f1_score:0.824751
[290]	train-f1_score:0.824709
[291]	train-f1_score:0.824793
[292]	train-f1_score:0.824835
[293]	train-f1_score:0.824835
[294]	train-f1_score:0.82496
[295]	train-f1_score:0.824918
[296]	train-f1_score:0.825002
[297]	train-f1_score:0.824919
[298]	train-f1_score:0.825044
[299]	train-f1_score:0.825044
[300]	train-f1_score:0.825002
[301]	train-f1_score:0.825128
[302]	train-f1_score:0.825379
[303]	train-f1_score:0.825379
[304]	train-f1_score:0.82542
[305]	train-f1_score:0.82542
[306]	train-f1_score:0.825504
[307]	train-f1_score:0.825546
[308]	train-f1_score:0.825587
[309]	train-f1_score:0.825587
[310]	train-f1_score:0.825504
[311]	train-f1_sc

[553]	train-f1_score:0.834615
[554]	train-f1_score:0.834531
[555]	train-f1_score:0.834531
[556]	train-f1_score:0.834531
[557]	train-f1_score:0.834657
[558]	train-f1_score:0.834573
[559]	train-f1_score:0.835032
[560]	train-f1_score:0.835032
[561]	train-f1_score:0.835074
[562]	train-f1_score:0.835158
[563]	train-f1_score:0.835116
[564]	train-f1_score:0.835074
[565]	train-f1_score:0.835074
[566]	train-f1_score:0.835116
[567]	train-f1_score:0.835116
[568]	train-f1_score:0.835116
[569]	train-f1_score:0.835075
[570]	train-f1_score:0.835367
[571]	train-f1_score:0.835284
[572]	train-f1_score:0.8352
[573]	train-f1_score:0.835325
[574]	train-f1_score:0.835367
[575]	train-f1_score:0.835451
[576]	train-f1_score:0.835576
[577]	train-f1_score:0.83566
[578]	train-f1_score:0.835701
[579]	train-f1_score:0.835743
[580]	train-f1_score:0.835701
[581]	train-f1_score:0.835785
[582]	train-f1_score:0.835701
[583]	train-f1_score:0.835701
[584]	train-f1_score:0.835743
[585]	train-f1_score:0.835827
[586]	train-f

[827]	train-f1_score:0.845939
[828]	train-f1_score:0.845939
[829]	train-f1_score:0.846023
[830]	train-f1_score:0.845981
[831]	train-f1_score:0.846023
[832]	train-f1_score:0.846065
[833]	train-f1_score:0.846065
[834]	train-f1_score:0.846274
[835]	train-f1_score:0.846315
[836]	train-f1_score:0.846399
[837]	train-f1_score:0.846483
[838]	train-f1_score:0.846524
[839]	train-f1_score:0.846483
[840]	train-f1_score:0.846399
[841]	train-f1_score:0.846733
[842]	train-f1_score:0.846608
[843]	train-f1_score:0.84665
[844]	train-f1_score:0.846733
[845]	train-f1_score:0.846733
[846]	train-f1_score:0.8469
[847]	train-f1_score:0.847068
[848]	train-f1_score:0.847193
[849]	train-f1_score:0.847193
[850]	train-f1_score:0.847151
[851]	train-f1_score:0.84711
[852]	train-f1_score:0.847068
[853]	train-f1_score:0.846984
[854]	train-f1_score:0.847026
[855]	train-f1_score:0.846984
[856]	train-f1_score:0.846901
[857]	train-f1_score:0.846942
[858]	train-f1_score:0.846984
[859]	train-f1_score:0.846984
[860]	train-f1

In [75]:
f1_score_1

0.80790940644980958

In [71]:
model_2,feature_score_2,preds2_01,preds1_01_for_test,preds_2,preds_1_for_test = model_train_for_testset(
    random_seed = 1000,gamma = 0.1,max_depth = 7,lambd = 10,subsample = 0.7,colsample_bytree = 0.7,
    min_child_weight = 1.1,colsample_bylevel = 0.7,model_1 = model_1,dataset2 = dataset2,
    dataset3 = dataset3,num_boost_round_value = 800,threshold_value = 0.4)

[0]	train-f1_score:0.82335
[1]	train-f1_score:0.826583
[2]	train-f1_score:0.827527
[3]	train-f1_score:0.827806
[4]	train-f1_score:0.828438
[5]	train-f1_score:0.828002
[6]	train-f1_score:0.828286
[7]	train-f1_score:0.827853
[8]	train-f1_score:0.827846
[9]	train-f1_score:0.828368
[10]	train-f1_score:0.828151
[11]	train-f1_score:0.828344
[12]	train-f1_score:0.828265
[13]	train-f1_score:0.828433
[14]	train-f1_score:0.828222
[15]	train-f1_score:0.828573
[16]	train-f1_score:0.828524
[17]	train-f1_score:0.828516
[18]	train-f1_score:0.828688
[19]	train-f1_score:0.828926
[20]	train-f1_score:0.828949
[21]	train-f1_score:0.829031
[22]	train-f1_score:0.82903
[23]	train-f1_score:0.828627
[24]	train-f1_score:0.82924
[25]	train-f1_score:0.828997
[26]	train-f1_score:0.829082
[27]	train-f1_score:0.829005
[28]	train-f1_score:0.82903
[29]	train-f1_score:0.8293
[30]	train-f1_score:0.829084
[31]	train-f1_score:0.828869
[32]	train-f1_score:0.829323
[33]	train-f1_score:0.829352
[34]	train-f1_score:0.829618
[

[278]	train-f1_score:0.835423
[279]	train-f1_score:0.835315
[280]	train-f1_score:0.835315
[281]	train-f1_score:0.835315
[282]	train-f1_score:0.835315
[283]	train-f1_score:0.835422
[284]	train-f1_score:0.83553
[285]	train-f1_score:0.835476
[286]	train-f1_score:0.835423
[287]	train-f1_score:0.83537
[288]	train-f1_score:0.835396
[289]	train-f1_score:0.835423
[290]	train-f1_score:0.835397
[291]	train-f1_score:0.835344
[292]	train-f1_score:0.83545
[293]	train-f1_score:0.83545
[294]	train-f1_score:0.835583
[295]	train-f1_score:0.835636
[296]	train-f1_score:0.835879
[297]	train-f1_score:0.835906
[298]	train-f1_score:0.835878
[299]	train-f1_score:0.835797
[300]	train-f1_score:0.835798
[301]	train-f1_score:0.835798
[302]	train-f1_score:0.835797
[303]	train-f1_score:0.835876
[304]	train-f1_score:0.835877
[305]	train-f1_score:0.835931
[306]	train-f1_score:0.835824
[307]	train-f1_score:0.835984
[308]	train-f1_score:0.836091
[309]	train-f1_score:0.836144
[310]	train-f1_score:0.836118
[311]	train-f1

[553]	train-f1_score:0.841109
[554]	train-f1_score:0.841163
[555]	train-f1_score:0.841217
[556]	train-f1_score:0.841164
[557]	train-f1_score:0.841164
[558]	train-f1_score:0.841245
[559]	train-f1_score:0.841272
[560]	train-f1_score:0.841326
[561]	train-f1_score:0.841378
[562]	train-f1_score:0.841352
[563]	train-f1_score:0.841352
[564]	train-f1_score:0.841325
[565]	train-f1_score:0.841405
[566]	train-f1_score:0.841592
[567]	train-f1_score:0.841538
[568]	train-f1_score:0.841512
[569]	train-f1_score:0.841485
[570]	train-f1_score:0.841513
[571]	train-f1_score:0.841353
[572]	train-f1_score:0.841461
[573]	train-f1_score:0.841488
[574]	train-f1_score:0.841461
[575]	train-f1_score:0.841596
[576]	train-f1_score:0.841623
[577]	train-f1_score:0.841515
[578]	train-f1_score:0.841569
[579]	train-f1_score:0.841703
[580]	train-f1_score:0.841703
[581]	train-f1_score:0.841703
[582]	train-f1_score:0.841648
[583]	train-f1_score:0.841621
[584]	train-f1_score:0.841675
[585]	train-f1_score:0.84173
[586]	train

In [76]:
feature_score_2

[('device_type', 3404),
 ('launch_day_near_pred', 1925),
 ('act_author_count_max_ratio', 1822),
 ('register_day', 1690),
 ('act_count_pred', 1553),
 ('activity_page_0', 1531),
 ('activity_page_3', 1529),
 ('action_type_1', 1528),
 ('act_author_count_average', 1522),
 ('action_type_0', 1494),
 ('act_author_count_std', 1492),
 ('activity_page_1', 1490),
 ('act_author_count_skew', 1397),
 ('act_count_average', 1396),
 ('activity_page_2', 1320),
 ('act_author_count_kurtosis', 1300),
 ('register_type', 1273),
 ('action_type_2', 1258),
 ('act_count_sum', 1252),
 ('act_count_diff_average', 1233),
 ('act_count_diff_sum', 1188),
 ('act_count_diff_plus_max', 1158),
 ('act_count_min', 1156),
 ('act_author_count_len', 1104),
 ('act_day_near_pred', 1060),
 ('act_count_max', 980),
 ('act_author_count_max', 958),
 ('act_count_diff_skew', 931),
 ('act_count_skew', 890),
 ('launch_day_sum_average', 854),
 ('launch_day_sum', 852),
 ('act_count_diff_minus_max', 849),
 ('act_count_day_max', 840),
 ('act_c

In [63]:
table15_30_pred['result'] = preds2_01
result = table15_30_pred[table15_30_pred['result'] == 1]
result = result[['user_id']]
result.shape

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(24130, 1)

In [64]:
result.to_csv('baseline2_xgb_600_0.4.csv',index = False, header = False) # 另一套参数

### 调参 

In [ ]:
只是简单试了下其他的参数，调参对于这个数据集规模影响不大

In [59]:
max_depth = [4,5,6,7,8,9,10]
f1_score_1_list = []

In [60]:
for i in range(len(max_depth)):
    model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
        random_seed = 1000,gamma = 0.1,max_depth = max_depth[i],lambd = 0,subsample = 0.95,colsample_bytree = 0.95,
        min_child_weight = 1,colsample_bylevel = 0.95,
        dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 800,threshold_value = 0.5)
    f1_score_1_list.append(f1_score_1)    

[0]	train-f1_score:0.792447
[1]	train-f1_score:0.795133
[2]	train-f1_score:0.79519
[3]	train-f1_score:0.795424
[4]	train-f1_score:0.798614
[5]	train-f1_score:0.798074
[6]	train-f1_score:0.798106
[7]	train-f1_score:0.797988
[8]	train-f1_score:0.797683
[9]	train-f1_score:0.797948
[10]	train-f1_score:0.797993
[11]	train-f1_score:0.797867
[12]	train-f1_score:0.797658
[13]	train-f1_score:0.797327
[14]	train-f1_score:0.798076
[15]	train-f1_score:0.797993
[16]	train-f1_score:0.798077
[17]	train-f1_score:0.797991
[18]	train-f1_score:0.798365
[19]	train-f1_score:0.798404
[20]	train-f1_score:0.797642
[21]	train-f1_score:0.797619
[22]	train-f1_score:0.79778
[23]	train-f1_score:0.798031
[24]	train-f1_score:0.797949
[25]	train-f1_score:0.797948
[26]	train-f1_score:0.797575
[27]	train-f1_score:0.797701
[28]	train-f1_score:0.797661
[29]	train-f1_score:0.797621
[30]	train-f1_score:0.797578
[31]	train-f1_score:0.797578
[32]	train-f1_score:0.797911
[33]	train-f1_score:0.797747
[34]	train-f1_score:0.7980

[278]	train-f1_score:0.80227
[279]	train-f1_score:0.80227
[280]	train-f1_score:0.802142
[281]	train-f1_score:0.8021
[282]	train-f1_score:0.802144
[283]	train-f1_score:0.802102
[284]	train-f1_score:0.802185
[285]	train-f1_score:0.802102
[286]	train-f1_score:0.802312
[287]	train-f1_score:0.802311
[288]	train-f1_score:0.802395
[289]	train-f1_score:0.802269
[290]	train-f1_score:0.802436
[291]	train-f1_score:0.802436
[292]	train-f1_score:0.802478
[293]	train-f1_score:0.802478
[294]	train-f1_score:0.802478
[295]	train-f1_score:0.802437
[296]	train-f1_score:0.802647
[297]	train-f1_score:0.802687
[298]	train-f1_score:0.802771
[299]	train-f1_score:0.802771
[300]	train-f1_score:0.802814
[301]	train-f1_score:0.802898
[302]	train-f1_score:0.802856
[303]	train-f1_score:0.80294
[304]	train-f1_score:0.802982
[305]	train-f1_score:0.802938
[306]	train-f1_score:0.803022
[307]	train-f1_score:0.803023
[308]	train-f1_score:0.803065
[309]	train-f1_score:0.802981
[310]	train-f1_score:0.80298
[311]	train-f1_s

[553]	train-f1_score:0.809361
[554]	train-f1_score:0.809319
[555]	train-f1_score:0.809278
[556]	train-f1_score:0.809236
[557]	train-f1_score:0.809277
[558]	train-f1_score:0.809236
[559]	train-f1_score:0.809362
[560]	train-f1_score:0.809362
[561]	train-f1_score:0.809362
[562]	train-f1_score:0.809446
[563]	train-f1_score:0.809488
[564]	train-f1_score:0.809488
[565]	train-f1_score:0.809572
[566]	train-f1_score:0.80974
[567]	train-f1_score:0.80974
[568]	train-f1_score:0.80974
[569]	train-f1_score:0.809782
[570]	train-f1_score:0.809698
[571]	train-f1_score:0.809698
[572]	train-f1_score:0.809699
[573]	train-f1_score:0.809657
[574]	train-f1_score:0.80974
[575]	train-f1_score:0.80974
[576]	train-f1_score:0.80974
[577]	train-f1_score:0.80974
[578]	train-f1_score:0.809782
[579]	train-f1_score:0.809824
[580]	train-f1_score:0.809824
[581]	train-f1_score:0.809824
[582]	train-f1_score:0.809782
[583]	train-f1_score:0.809824
[584]	train-f1_score:0.809865
[585]	train-f1_score:0.809739
[586]	train-f1_sc

[28]	train-f1_score:0.80144
[29]	train-f1_score:0.801609
[30]	train-f1_score:0.801654
[31]	train-f1_score:0.802154
[32]	train-f1_score:0.802236
[33]	train-f1_score:0.802441
[34]	train-f1_score:0.802614
[35]	train-f1_score:0.802278
[36]	train-f1_score:0.802191
[37]	train-f1_score:0.802359
[38]	train-f1_score:0.802316
[39]	train-f1_score:0.802486
[40]	train-f1_score:0.802356
[41]	train-f1_score:0.802272
[42]	train-f1_score:0.802229
[43]	train-f1_score:0.80231
[44]	train-f1_score:0.802103
[45]	train-f1_score:0.80244
[46]	train-f1_score:0.802397
[47]	train-f1_score:0.802768
[48]	train-f1_score:0.802517
[49]	train-f1_score:0.802642
[50]	train-f1_score:0.802811
[51]	train-f1_score:0.802937
[52]	train-f1_score:0.802566
[53]	train-f1_score:0.802736
[54]	train-f1_score:0.802523
[55]	train-f1_score:0.802391
[56]	train-f1_score:0.802307
[57]	train-f1_score:0.802472
[58]	train-f1_score:0.802642
[59]	train-f1_score:0.802567
[60]	train-f1_score:0.802524
[61]	train-f1_score:0.802525
[62]	train-f1_sco

[305]	train-f1_score:0.810438
[306]	train-f1_score:0.810354
[307]	train-f1_score:0.810564
[308]	train-f1_score:0.810521
[309]	train-f1_score:0.810563
[310]	train-f1_score:0.810607
[311]	train-f1_score:0.810606
[312]	train-f1_score:0.810607
[313]	train-f1_score:0.810691
[314]	train-f1_score:0.810691
[315]	train-f1_score:0.810775
[316]	train-f1_score:0.810859
[317]	train-f1_score:0.810775
[318]	train-f1_score:0.810984
[319]	train-f1_score:0.811027
[320]	train-f1_score:0.810985
[321]	train-f1_score:0.811068
[322]	train-f1_score:0.811067
[323]	train-f1_score:0.810942
[324]	train-f1_score:0.810984
[325]	train-f1_score:0.811193
[326]	train-f1_score:0.81115
[327]	train-f1_score:0.811319
[328]	train-f1_score:0.811318
[329]	train-f1_score:0.81136
[330]	train-f1_score:0.811362
[331]	train-f1_score:0.811403
[332]	train-f1_score:0.811487
[333]	train-f1_score:0.811486
[334]	train-f1_score:0.811528
[335]	train-f1_score:0.811528
[336]	train-f1_score:0.811612
[337]	train-f1_score:0.81178
[338]	train-f

...

[411]	train-f1_score:0.900335
[412]	train-f1_score:0.900461
[413]	train-f1_score:0.900461
[414]	train-f1_score:0.900672
[415]	train-f1_score:0.901008
[416]	train-f1_score:0.901133
[417]	train-f1_score:0.901133
[418]	train-f1_score:0.901008
[419]	train-f1_score:0.901092
[420]	train-f1_score:0.901301
[421]	train-f1_score:0.901468
[422]	train-f1_score:0.901426
[423]	train-f1_score:0.901594
[424]	train-f1_score:0.901635
[425]	train-f1_score:0.901803
[426]	train-f1_score:0.901846
[427]	train-f1_score:0.901888
[428]	train-f1_score:0.901846
[429]	train-f1_score:0.902014
[430]	train-f1_score:0.902181
[431]	train-f1_score:0.902307
[432]	train-f1_score:0.902349
[433]	train-f1_score:0.902475
[434]	train-f1_score:0.902726
[435]	train-f1_score:0.902809
[436]	train-f1_score:0.902935
[437]	train-f1_score:0.902852
[438]	train-f1_score:0.902894
[439]	train-f1_score:0.902852
[440]	train-f1_score:0.902852
[441]	train-f1_score:0.902852
[442]	train-f1_score:0.903061
[443]	train-f1_score:0.903103
[444]	trai

[685]	train-f1_score:0.917655
[686]	train-f1_score:0.917612
[687]	train-f1_score:0.917486
[688]	train-f1_score:0.917654
[689]	train-f1_score:0.917737
[690]	train-f1_score:0.917863
[691]	train-f1_score:0.917947
[692]	train-f1_score:0.917947
[693]	train-f1_score:0.917947
[694]	train-f1_score:0.917989
[695]	train-f1_score:0.918241
[696]	train-f1_score:0.918241
[697]	train-f1_score:0.918199
[698]	train-f1_score:0.918325
[699]	train-f1_score:0.918577
[700]	train-f1_score:0.918618
[701]	train-f1_score:0.918702
[702]	train-f1_score:0.918744
[703]	train-f1_score:0.918785
[704]	train-f1_score:0.918911
[705]	train-f1_score:0.918995
[706]	train-f1_score:0.919079
[707]	train-f1_score:0.919373
[708]	train-f1_score:0.919499
[709]	train-f1_score:0.919499
[710]	train-f1_score:0.919416
[711]	train-f1_score:0.919458
[712]	train-f1_score:0.919584
[713]	train-f1_score:0.919584
[714]	train-f1_score:0.919668
[715]	train-f1_score:0.919668
[716]	train-f1_score:0.91971
[717]	train-f1_score:0.919794
[718]	train

[162]	train-f1_score:0.899013
[163]	train-f1_score:0.899055
[164]	train-f1_score:0.8996
[165]	train-f1_score:0.899853
[166]	train-f1_score:0.89981
[167]	train-f1_score:0.899935
[168]	train-f1_score:0.900103
[169]	train-f1_score:0.900229
[170]	train-f1_score:0.900858
[171]	train-f1_score:0.900984
[172]	train-f1_score:0.901318
[173]	train-f1_score:0.901276
[174]	train-f1_score:0.901444
[175]	train-f1_score:0.901695
[176]	train-f1_score:0.901737
[177]	train-f1_score:0.901779
[178]	train-f1_score:0.901989
[179]	train-f1_score:0.902241
[180]	train-f1_score:0.902325
[181]	train-f1_score:0.902869
[182]	train-f1_score:0.902869
[183]	train-f1_score:0.903036
[184]	train-f1_score:0.903079
[185]	train-f1_score:0.903331
[186]	train-f1_score:0.903623
[187]	train-f1_score:0.903496
[188]	train-f1_score:0.903496
[189]	train-f1_score:0.903539
[190]	train-f1_score:0.903748
[191]	train-f1_score:0.903622
[192]	train-f1_score:0.903789
[193]	train-f1_score:0.903832
[194]	train-f1_score:0.903998
[195]	train-f

[437]	train-f1_score:0.928479
[438]	train-f1_score:0.928521
[439]	train-f1_score:0.928521
[440]	train-f1_score:0.928605
[441]	train-f1_score:0.928647
[442]	train-f1_score:0.928689
[443]	train-f1_score:0.928689
[444]	train-f1_score:0.928605
[445]	train-f1_score:0.928647
[446]	train-f1_score:0.928689
[447]	train-f1_score:0.928688
[448]	train-f1_score:0.928856
[449]	train-f1_score:0.92894
[450]	train-f1_score:0.929107
[451]	train-f1_score:0.929107
[452]	train-f1_score:0.929233
[453]	train-f1_score:0.929317
[454]	train-f1_score:0.929317
[455]	train-f1_score:0.929401
[456]	train-f1_score:0.929485
[457]	train-f1_score:0.929443
[458]	train-f1_score:0.929485
[459]	train-f1_score:0.929485
[460]	train-f1_score:0.929569
[461]	train-f1_score:0.929778
[462]	train-f1_score:0.929736
[463]	train-f1_score:0.929736
[464]	train-f1_score:0.929736
[465]	train-f1_score:0.929862
[466]	train-f1_score:0.929904
[467]	train-f1_score:0.929904
[468]	train-f1_score:0.930029
[469]	train-f1_score:0.930029
[470]	train

[712]	train-f1_score:0.94287
[713]	train-f1_score:0.94287
[714]	train-f1_score:0.942912
[715]	train-f1_score:0.942912
[716]	train-f1_score:0.942996
[717]	train-f1_score:0.943121
[718]	train-f1_score:0.943205
[719]	train-f1_score:0.943205
[720]	train-f1_score:0.943247
[721]	train-f1_score:0.943289
[722]	train-f1_score:0.943457
[723]	train-f1_score:0.943457
[724]	train-f1_score:0.943583
[725]	train-f1_score:0.943541
[726]	train-f1_score:0.943583
[727]	train-f1_score:0.943793
[728]	train-f1_score:0.94396
[729]	train-f1_score:0.94396
[730]	train-f1_score:0.94396
[731]	train-f1_score:0.944044
[732]	train-f1_score:0.944212
[733]	train-f1_score:0.944254
[734]	train-f1_score:0.944296
[735]	train-f1_score:0.944296
[736]	train-f1_score:0.944296
[737]	train-f1_score:0.944296
[738]	train-f1_score:0.944296
[739]	train-f1_score:0.944296
[740]	train-f1_score:0.944422
[741]	train-f1_score:0.944464
[742]	train-f1_score:0.944591
[743]	train-f1_score:0.944716
[744]	train-f1_score:0.9448
[745]	train-f1_sc

In [ ]:
200时的结果，7最合适（429）

In [51]:
f1_score_1_list

[0.81235242487184878,
 0.81355466657971842,
 0.81411751101731622,
 0.8142985347156072,
 0.81380147432834105,
 0.81196622170783006,
 0.81117683667550755]

In [ ]:
400时的结果，7和8都可以，且都比200时要好（441）

In [53]:
f1_score_1_list

[0.81235242487184878,
 0.81355466657971842,
 0.81411751101731622,
 0.8142985347156072,
 0.81380147432834105,
 0.81196622170783006,
 0.81117683667550755,
 0.81359824814399406,
 0.81405721946122145,
 0.81433335335169077,
 0.81441217577859015,
 0.81441581798755591,
 0.81032243322155062,
 0.81001420767969667]

In [ ]:
600时，5最好（507），6（474）

In [58]:
f1_score_1_list

[0.81502393486895441,
 0.81507469048558412,
 0.81474184779584957,
 0.81426052613174771,
 0.81397103106692714,
 0.80974510769894203,
 0.80892588063103765]

In [ ]:
800时，5最好（492），但是已经开始降了

In [61]:
f1_score_1_list

[0.81491972230715259,
 0.81492299133367729,
 0.81464036807424756,
 0.81461317885467233,
 0.81291746193407166,
 0.80903149065830271,
 0.80816024423697308]

In [ ]:
选择5

In [185]:
gamma = [ i/100 for i in range(0,20,1)]
f1_score_1_list = []

In [187]:
for i in range(len(gamma)):
    model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = gamma[i],max_depth = 5,lambd = 0,subsample = 0.95,colsample_bytree = 0.95,
    min_child_weight = 1,colsample_bylevel = 0.95,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 200,threshold_value = 0.7)
    f1_score_1_list.append(f1_score_1)   

[0]	train-f1_score:0.797637
[1]	train-f1_score:0.799805
[2]	train-f1_score:0.800713
[3]	train-f1_score:0.799812
[4]	train-f1_score:0.799436
[5]	train-f1_score:0.799708
[6]	train-f1_score:0.800129
[7]	train-f1_score:0.800012
[8]	train-f1_score:0.799991
[9]	train-f1_score:0.800468
[10]	train-f1_score:0.800626
[11]	train-f1_score:0.800458
[12]	train-f1_score:0.80047
[13]	train-f1_score:0.800685
[14]	train-f1_score:0.801091
[15]	train-f1_score:0.800589
[16]	train-f1_score:0.800545
[17]	train-f1_score:0.800386
[18]	train-f1_score:0.800552
[19]	train-f1_score:0.80035
[20]	train-f1_score:0.800726
[21]	train-f1_score:0.800927
[22]	train-f1_score:0.801057
[23]	train-f1_score:0.801141
[24]	train-f1_score:0.801055
[25]	train-f1_score:0.80089
[26]	train-f1_score:0.801099
[27]	train-f1_score:0.801432
[28]	train-f1_score:0.801177
[29]	train-f1_score:0.801262
[30]	train-f1_score:0.801796
[31]	train-f1_score:0.80171
[32]	train-f1_score:0.801836
[33]	train-f1_score:0.801963
[34]	train-f1_score:0.802462

[80]	train-f1_score:0.802212
[81]	train-f1_score:0.802252
[82]	train-f1_score:0.802546
[83]	train-f1_score:0.80225
[84]	train-f1_score:0.802419
[85]	train-f1_score:0.802464
[86]	train-f1_score:0.8023
[87]	train-f1_score:0.802556
[88]	train-f1_score:0.802433
[89]	train-f1_score:0.802433
[90]	train-f1_score:0.802435
[91]	train-f1_score:0.802227
[92]	train-f1_score:0.802397
[93]	train-f1_score:0.802148
[94]	train-f1_score:0.80244
[95]	train-f1_score:0.802528
[96]	train-f1_score:0.80236
[97]	train-f1_score:0.802193
[98]	train-f1_score:0.802363
[99]	train-f1_score:0.80249
[100]	train-f1_score:0.802321
[101]	train-f1_score:0.802366
[102]	train-f1_score:0.802449
[103]	train-f1_score:0.802449
[104]	train-f1_score:0.802407
[105]	train-f1_score:0.802617
[106]	train-f1_score:0.802618
[107]	train-f1_score:0.802702
[108]	train-f1_score:0.802747
[109]	train-f1_score:0.803082
[110]	train-f1_score:0.803251
[111]	train-f1_score:0.803336
[112]	train-f1_score:0.803379
[113]	train-f1_score:0.803505
[114]	

[158]	train-f1_score:0.804976
[159]	train-f1_score:0.804977
[160]	train-f1_score:0.805018
[161]	train-f1_score:0.805186
[162]	train-f1_score:0.805102
[163]	train-f1_score:0.805271
[164]	train-f1_score:0.805229
[165]	train-f1_score:0.805271
[166]	train-f1_score:0.805271
[167]	train-f1_score:0.805396
[168]	train-f1_score:0.805355
[169]	train-f1_score:0.805481
[170]	train-f1_score:0.805354
[171]	train-f1_score:0.805648
[172]	train-f1_score:0.80548
[173]	train-f1_score:0.805565
[174]	train-f1_score:0.805607
[175]	train-f1_score:0.805648
[176]	train-f1_score:0.805734
[177]	train-f1_score:0.805901
[178]	train-f1_score:0.805734
[179]	train-f1_score:0.80565
[180]	train-f1_score:0.805984
[181]	train-f1_score:0.805899
[182]	train-f1_score:0.805902
[183]	train-f1_score:0.805817
[184]	train-f1_score:0.805692
[185]	train-f1_score:0.805818
[186]	train-f1_score:0.805944
[187]	train-f1_score:0.805986
[188]	train-f1_score:0.805985
[189]	train-f1_score:0.805901
[190]	train-f1_score:0.806028
[191]	train-

[36]	train-f1_score:0.802294
[37]	train-f1_score:0.802418
[38]	train-f1_score:0.802251
[39]	train-f1_score:0.802498
[40]	train-f1_score:0.802581
[41]	train-f1_score:0.802624
[42]	train-f1_score:0.802288
[43]	train-f1_score:0.80262
[44]	train-f1_score:0.802619
[45]	train-f1_score:0.802528
[46]	train-f1_score:0.802487
[47]	train-f1_score:0.802521
[48]	train-f1_score:0.802562
[49]	train-f1_score:0.802686
[50]	train-f1_score:0.802853
[51]	train-f1_score:0.802725
[52]	train-f1_score:0.802895
[53]	train-f1_score:0.802682
[54]	train-f1_score:0.802809
[55]	train-f1_score:0.803015
[56]	train-f1_score:0.802852
[57]	train-f1_score:0.802723
[58]	train-f1_score:0.802553
[59]	train-f1_score:0.802507
[60]	train-f1_score:0.802506
[61]	train-f1_score:0.802802
[62]	train-f1_score:0.802758
[63]	train-f1_score:0.802671
[64]	train-f1_score:0.80263
[65]	train-f1_score:0.802504
[66]	train-f1_score:0.802582
[67]	train-f1_score:0.8025
[68]	train-f1_score:0.802704
[69]	train-f1_score:0.802624
[70]	train-f1_scor

[115]	train-f1_score:0.803546
[116]	train-f1_score:0.803505
[117]	train-f1_score:0.803377
[118]	train-f1_score:0.803377
[119]	train-f1_score:0.803547
[120]	train-f1_score:0.80363
[121]	train-f1_score:0.803463
[122]	train-f1_score:0.803463
[123]	train-f1_score:0.803336
[124]	train-f1_score:0.803716
[125]	train-f1_score:0.803717
[126]	train-f1_score:0.803759
[127]	train-f1_score:0.803885
[128]	train-f1_score:0.803843
[129]	train-f1_score:0.803844
[130]	train-f1_score:0.804052
[131]	train-f1_score:0.804013
[132]	train-f1_score:0.804095
[133]	train-f1_score:0.804222
[134]	train-f1_score:0.804181
[135]	train-f1_score:0.804265
[136]	train-f1_score:0.804349
[137]	train-f1_score:0.804306
[138]	train-f1_score:0.804347
[139]	train-f1_score:0.804516
[140]	train-f1_score:0.804642
[141]	train-f1_score:0.8046
[142]	train-f1_score:0.804683
[143]	train-f1_score:0.804767
[144]	train-f1_score:0.804725
[145]	train-f1_score:0.804766
[146]	train-f1_score:0.804725
[147]	train-f1_score:0.804851
[148]	train-f

...

[183]	train-f1_score:0.805817
[184]	train-f1_score:0.805692
[185]	train-f1_score:0.805818
[186]	train-f1_score:0.805944
[187]	train-f1_score:0.805986
[188]	train-f1_score:0.805985
[189]	train-f1_score:0.805901
[190]	train-f1_score:0.806028
[191]	train-f1_score:0.806027
[192]	train-f1_score:0.805944
[193]	train-f1_score:0.805819
[194]	train-f1_score:0.805861
[195]	train-f1_score:0.805989
[196]	train-f1_score:0.805905
[197]	train-f1_score:0.805862
[198]	train-f1_score:0.806029
[199]	train-f1_score:0.805945
model1的：1的个数和全部的个数
19915 37446
[0]	train-f1_score:0.797637
[1]	train-f1_score:0.799805
[2]	train-f1_score:0.800713
[3]	train-f1_score:0.799812
[4]	train-f1_score:0.799436
[5]	train-f1_score:0.799708
[6]	train-f1_score:0.800129
[7]	train-f1_score:0.800012
[8]	train-f1_score:0.799991
[9]	train-f1_score:0.800468
[10]	train-f1_score:0.800626
[11]	train-f1_score:0.800458
[12]	train-f1_score:0.80047
[13]	train-f1_score:0.800685
[14]	train-f1_score:0.801091
[15]	train-f1_score:0.800589
[16]	t

[61]	train-f1_score:0.802802
[62]	train-f1_score:0.802758
[63]	train-f1_score:0.802671
[64]	train-f1_score:0.80263
[65]	train-f1_score:0.802504
[66]	train-f1_score:0.802582
[67]	train-f1_score:0.8025
[68]	train-f1_score:0.802704
[69]	train-f1_score:0.802624
[70]	train-f1_score:0.802501
[71]	train-f1_score:0.802666
[72]	train-f1_score:0.802497
[73]	train-f1_score:0.802456
[74]	train-f1_score:0.802334
[75]	train-f1_score:0.802379
[76]	train-f1_score:0.802254
[77]	train-f1_score:0.802385
[78]	train-f1_score:0.802299
[79]	train-f1_score:0.802339
[80]	train-f1_score:0.802212
[81]	train-f1_score:0.802252
[82]	train-f1_score:0.802546
[83]	train-f1_score:0.80225
[84]	train-f1_score:0.802419
[85]	train-f1_score:0.802464
[86]	train-f1_score:0.8023
[87]	train-f1_score:0.802556
[88]	train-f1_score:0.802433
[89]	train-f1_score:0.802433
[90]	train-f1_score:0.802435
[91]	train-f1_score:0.802227
[92]	train-f1_score:0.802397
[93]	train-f1_score:0.802148
[94]	train-f1_score:0.80244
[95]	train-f1_score:0

[140]	train-f1_score:0.804642
[141]	train-f1_score:0.8046
[142]	train-f1_score:0.804683
[143]	train-f1_score:0.804767
[144]	train-f1_score:0.804725
[145]	train-f1_score:0.804766
[146]	train-f1_score:0.804725
[147]	train-f1_score:0.804851
[148]	train-f1_score:0.804767
[149]	train-f1_score:0.804725
[150]	train-f1_score:0.804976
[151]	train-f1_score:0.804933
[152]	train-f1_score:0.805017
[153]	train-f1_score:0.80527
[154]	train-f1_score:0.805059
[155]	train-f1_score:0.804975
[156]	train-f1_score:0.805144
[157]	train-f1_score:0.805059
[158]	train-f1_score:0.804976
[159]	train-f1_score:0.804977
[160]	train-f1_score:0.805018
[161]	train-f1_score:0.805186
[162]	train-f1_score:0.805102
[163]	train-f1_score:0.805271
[164]	train-f1_score:0.805229
[165]	train-f1_score:0.805271
[166]	train-f1_score:0.805271
[167]	train-f1_score:0.805396
[168]	train-f1_score:0.805355
[169]	train-f1_score:0.805481
[170]	train-f1_score:0.805354
[171]	train-f1_score:0.805648
[172]	train-f1_score:0.80548
[173]	train-f1

[17]	train-f1_score:0.800386
[18]	train-f1_score:0.800552
[19]	train-f1_score:0.80035
[20]	train-f1_score:0.800726
[21]	train-f1_score:0.800927
[22]	train-f1_score:0.801057
[23]	train-f1_score:0.801141
[24]	train-f1_score:0.801055
[25]	train-f1_score:0.80089
[26]	train-f1_score:0.801099
[27]	train-f1_score:0.801432
[28]	train-f1_score:0.801177
[29]	train-f1_score:0.801262
[30]	train-f1_score:0.801796
[31]	train-f1_score:0.80171
[32]	train-f1_score:0.801836
[33]	train-f1_score:0.801963
[34]	train-f1_score:0.802462
[35]	train-f1_score:0.802506
[36]	train-f1_score:0.802294
[37]	train-f1_score:0.802418
[38]	train-f1_score:0.802251
[39]	train-f1_score:0.802498
[40]	train-f1_score:0.802581
[41]	train-f1_score:0.802624
[42]	train-f1_score:0.802288
[43]	train-f1_score:0.80262
[44]	train-f1_score:0.802619
[45]	train-f1_score:0.802528
[46]	train-f1_score:0.802487
[47]	train-f1_score:0.802521
[48]	train-f1_score:0.802562
[49]	train-f1_score:0.802686
[50]	train-f1_score:0.802853
[51]	train-f1_scor

[97]	train-f1_score:0.802193
[98]	train-f1_score:0.802363
[99]	train-f1_score:0.80249
[100]	train-f1_score:0.802321
[101]	train-f1_score:0.802366
[102]	train-f1_score:0.802449
[103]	train-f1_score:0.802449
[104]	train-f1_score:0.802407
[105]	train-f1_score:0.802617
[106]	train-f1_score:0.802618
[107]	train-f1_score:0.802702
[108]	train-f1_score:0.802747
[109]	train-f1_score:0.803082
[110]	train-f1_score:0.803251
[111]	train-f1_score:0.803336
[112]	train-f1_score:0.803379
[113]	train-f1_score:0.803505
[114]	train-f1_score:0.803547
[115]	train-f1_score:0.803546
[116]	train-f1_score:0.803505
[117]	train-f1_score:0.803377
[118]	train-f1_score:0.803377
[119]	train-f1_score:0.803547
[120]	train-f1_score:0.80363
[121]	train-f1_score:0.803463
[122]	train-f1_score:0.803463
[123]	train-f1_score:0.803336
[124]	train-f1_score:0.803716
[125]	train-f1_score:0.803717
[126]	train-f1_score:0.803759
[127]	train-f1_score:0.803885
[128]	train-f1_score:0.803843
[129]	train-f1_score:0.803844
[130]	train-f1_

In [ ]:
选择0.8

In [189]:
lambd = [ i/10 for i in range(0,100,5)]
f1_score_1_list = []

In [190]:
for i in range(len(lambd)):
    model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = 0.8,max_depth = 5,lambd = lambd[i],subsample = 0.95,colsample_bytree = 0.95,
    min_child_weight = 1,colsample_bylevel = 0.95,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 200,threshold_value = 0.7)
    f1_score_1_list.append(f1_score_1)   

[0]	train-f1_score:0.797637
[1]	train-f1_score:0.799805
[2]	train-f1_score:0.800713
[3]	train-f1_score:0.799812
[4]	train-f1_score:0.799436
[5]	train-f1_score:0.799708
[6]	train-f1_score:0.800129
[7]	train-f1_score:0.800012
[8]	train-f1_score:0.799991
[9]	train-f1_score:0.800468
[10]	train-f1_score:0.800626
[11]	train-f1_score:0.800458
[12]	train-f1_score:0.80047
[13]	train-f1_score:0.800685
[14]	train-f1_score:0.801091
[15]	train-f1_score:0.800589
[16]	train-f1_score:0.800545
[17]	train-f1_score:0.800386
[18]	train-f1_score:0.800552
[19]	train-f1_score:0.80035
[20]	train-f1_score:0.800726
[21]	train-f1_score:0.800927
[22]	train-f1_score:0.801057
[23]	train-f1_score:0.801141
[24]	train-f1_score:0.801055
[25]	train-f1_score:0.80089
[26]	train-f1_score:0.801099
[27]	train-f1_score:0.801432
[28]	train-f1_score:0.801177
[29]	train-f1_score:0.801262
[30]	train-f1_score:0.801796
[31]	train-f1_score:0.80171
[32]	train-f1_score:0.801836
[33]	train-f1_score:0.801963
[34]	train-f1_score:0.802462

[80]	train-f1_score:0.802256
[81]	train-f1_score:0.802255
[82]	train-f1_score:0.802257
[83]	train-f1_score:0.802297
[84]	train-f1_score:0.802176
[85]	train-f1_score:0.802095
[86]	train-f1_score:0.801968
[87]	train-f1_score:0.802098
[88]	train-f1_score:0.802098
[89]	train-f1_score:0.802059
[90]	train-f1_score:0.802101
[91]	train-f1_score:0.80223
[92]	train-f1_score:0.802527
[93]	train-f1_score:0.802492
[94]	train-f1_score:0.802488
[95]	train-f1_score:0.802617
[96]	train-f1_score:0.802571
[97]	train-f1_score:0.8027
[98]	train-f1_score:0.802827
[99]	train-f1_score:0.802829
[100]	train-f1_score:0.802787
[101]	train-f1_score:0.802747
[102]	train-f1_score:0.802661
[103]	train-f1_score:0.80283
[104]	train-f1_score:0.802787
[105]	train-f1_score:0.802831
[106]	train-f1_score:0.80262
[107]	train-f1_score:0.802955
[108]	train-f1_score:0.80279
[109]	train-f1_score:0.802916
[110]	train-f1_score:0.802874
[111]	train-f1_score:0.803042
[112]	train-f1_score:0.803044
[113]	train-f1_score:0.803129
[114]	

[158]	train-f1_score:0.804726
[159]	train-f1_score:0.804686
[160]	train-f1_score:0.804602
[161]	train-f1_score:0.804728
[162]	train-f1_score:0.804684
[163]	train-f1_score:0.8046
[164]	train-f1_score:0.804684
[165]	train-f1_score:0.8046
[166]	train-f1_score:0.804726
[167]	train-f1_score:0.80481
[168]	train-f1_score:0.804811
[169]	train-f1_score:0.804812
[170]	train-f1_score:0.804896
[171]	train-f1_score:0.804936
[172]	train-f1_score:0.80502
[173]	train-f1_score:0.805021
[174]	train-f1_score:0.805105
[175]	train-f1_score:0.805105
[176]	train-f1_score:0.805189
[177]	train-f1_score:0.805189
[178]	train-f1_score:0.805148
[179]	train-f1_score:0.805231
[180]	train-f1_score:0.805315
[181]	train-f1_score:0.805273
[182]	train-f1_score:0.805234
[183]	train-f1_score:0.805233
[184]	train-f1_score:0.805275
[185]	train-f1_score:0.805359
[186]	train-f1_score:0.805317
[187]	train-f1_score:0.8054
[188]	train-f1_score:0.805526
[189]	train-f1_score:0.805568
[190]	train-f1_score:0.805693
[191]	train-f1_sco

[36]	train-f1_score:0.801786
[37]	train-f1_score:0.801994
[38]	train-f1_score:0.801743
[39]	train-f1_score:0.801619
[40]	train-f1_score:0.801956
[41]	train-f1_score:0.802293
[42]	train-f1_score:0.801997
[43]	train-f1_score:0.801953
[44]	train-f1_score:0.801951
[45]	train-f1_score:0.80195
[46]	train-f1_score:0.801859
[47]	train-f1_score:0.802072
[48]	train-f1_score:0.80207
[49]	train-f1_score:0.80224
[50]	train-f1_score:0.802195
[51]	train-f1_score:0.802109
[52]	train-f1_score:0.80232
[53]	train-f1_score:0.802067
[54]	train-f1_score:0.802275
[55]	train-f1_score:0.80211
[56]	train-f1_score:0.802279
[57]	train-f1_score:0.802315
[58]	train-f1_score:0.802435
[59]	train-f1_score:0.802602
[60]	train-f1_score:0.802392
[61]	train-f1_score:0.802351
[62]	train-f1_score:0.802432
[63]	train-f1_score:0.80239
[64]	train-f1_score:0.802262
[65]	train-f1_score:0.802009
[66]	train-f1_score:0.802219
[67]	train-f1_score:0.802386
[68]	train-f1_score:0.802299
[69]	train-f1_score:0.802174
[70]	train-f1_score:

[116]	train-f1_score:0.802795
[117]	train-f1_score:0.802669
[118]	train-f1_score:0.802669
[119]	train-f1_score:0.802962
[120]	train-f1_score:0.802835
[121]	train-f1_score:0.803004
[122]	train-f1_score:0.802962
[123]	train-f1_score:0.802921
[124]	train-f1_score:0.80313
[125]	train-f1_score:0.803089
[126]	train-f1_score:0.803214
[127]	train-f1_score:0.803131
[128]	train-f1_score:0.803342
[129]	train-f1_score:0.803426
[130]	train-f1_score:0.803468
[131]	train-f1_score:0.803594
[132]	train-f1_score:0.803428
[133]	train-f1_score:0.80347
[134]	train-f1_score:0.803724
[135]	train-f1_score:0.803766
[136]	train-f1_score:0.803514
[137]	train-f1_score:0.80364
[138]	train-f1_score:0.803679
[139]	train-f1_score:0.803765
[140]	train-f1_score:0.803723
[141]	train-f1_score:0.803808
[142]	train-f1_score:0.803681
[143]	train-f1_score:0.803681
[144]	train-f1_score:0.803722
[145]	train-f1_score:0.80364
[146]	train-f1_score:0.803724
[147]	train-f1_score:0.803892
[148]	train-f1_score:0.803809
[149]	train-f1

...

[184]	train-f1_score:0.804437
[185]	train-f1_score:0.804437
[186]	train-f1_score:0.804479
[187]	train-f1_score:0.804395
[188]	train-f1_score:0.804479
[189]	train-f1_score:0.804395
[190]	train-f1_score:0.804395
[191]	train-f1_score:0.804521
[192]	train-f1_score:0.804562
[193]	train-f1_score:0.804605
[194]	train-f1_score:0.80469
[195]	train-f1_score:0.80469
[196]	train-f1_score:0.804732
[197]	train-f1_score:0.804691
[198]	train-f1_score:0.804774
[199]	train-f1_score:0.804732
model1的：1的个数和全部的个数
19961 37446
[0]	train-f1_score:0.798257
[1]	train-f1_score:0.799716
[2]	train-f1_score:0.800655
[3]	train-f1_score:0.800684
[4]	train-f1_score:0.799562
[5]	train-f1_score:0.79985
[6]	train-f1_score:0.800403
[7]	train-f1_score:0.800444
[8]	train-f1_score:0.799689
[9]	train-f1_score:0.800322
[10]	train-f1_score:0.800967
[11]	train-f1_score:0.800848
[12]	train-f1_score:0.801535
[13]	train-f1_score:0.801707
[14]	train-f1_score:0.800968
[15]	train-f1_score:0.801216
[16]	train-f1_score:0.801028
[17]	trai

[63]	train-f1_score:0.802224
[64]	train-f1_score:0.80214
[65]	train-f1_score:0.802351
[66]	train-f1_score:0.802346
[67]	train-f1_score:0.802391
[68]	train-f1_score:0.801972
[69]	train-f1_score:0.802268
[70]	train-f1_score:0.802146
[71]	train-f1_score:0.802102
[72]	train-f1_score:0.801977
[73]	train-f1_score:0.801813
[74]	train-f1_score:0.801852
[75]	train-f1_score:0.80189
[76]	train-f1_score:0.80172
[77]	train-f1_score:0.801975
[78]	train-f1_score:0.801764
[79]	train-f1_score:0.80172
[80]	train-f1_score:0.801512
[81]	train-f1_score:0.801888
[82]	train-f1_score:0.8021
[83]	train-f1_score:0.802015
[84]	train-f1_score:0.802062
[85]	train-f1_score:0.801979
[86]	train-f1_score:0.801852
[87]	train-f1_score:0.801982
[88]	train-f1_score:0.801856
[89]	train-f1_score:0.801772
[90]	train-f1_score:0.801772
[91]	train-f1_score:0.801856
[92]	train-f1_score:0.801984
[93]	train-f1_score:0.80211
[94]	train-f1_score:0.802194
[95]	train-f1_score:0.802321
[96]	train-f1_score:0.802196
[97]	train-f1_score:0

[141]	train-f1_score:0.803681
[142]	train-f1_score:0.803766
[143]	train-f1_score:0.803809
[144]	train-f1_score:0.803808
[145]	train-f1_score:0.803724
[146]	train-f1_score:0.803556
[147]	train-f1_score:0.803766
[148]	train-f1_score:0.803641
[149]	train-f1_score:0.803682
[150]	train-f1_score:0.80385
[151]	train-f1_score:0.803597
[152]	train-f1_score:0.803807
[153]	train-f1_score:0.803807
[154]	train-f1_score:0.803933
[155]	train-f1_score:0.803891
[156]	train-f1_score:0.803891
[157]	train-f1_score:0.804016
[158]	train-f1_score:0.80389
[159]	train-f1_score:0.803932
[160]	train-f1_score:0.804101
[161]	train-f1_score:0.804142
[162]	train-f1_score:0.804017
[163]	train-f1_score:0.804101
[164]	train-f1_score:0.80406
[165]	train-f1_score:0.804101
[166]	train-f1_score:0.80406
[167]	train-f1_score:0.804018
[168]	train-f1_score:0.804145
[169]	train-f1_score:0.804101
[170]	train-f1_score:0.804227
[171]	train-f1_score:0.804396
[172]	train-f1_score:0.804227
[173]	train-f1_score:0.804227
[174]	train-f1

[18]	train-f1_score:0.800952
[19]	train-f1_score:0.800961
[20]	train-f1_score:0.800753
[21]	train-f1_score:0.801076
[22]	train-f1_score:0.80079
[23]	train-f1_score:0.80117
[24]	train-f1_score:0.801377
[25]	train-f1_score:0.801832
[26]	train-f1_score:0.801956
[27]	train-f1_score:0.801881
[28]	train-f1_score:0.801579
[29]	train-f1_score:0.801832
[30]	train-f1_score:0.801449
[31]	train-f1_score:0.801577
[32]	train-f1_score:0.801709
[33]	train-f1_score:0.801709
[34]	train-f1_score:0.801794
[35]	train-f1_score:0.80183
[36]	train-f1_score:0.801915
[37]	train-f1_score:0.802248
[38]	train-f1_score:0.80204
[39]	train-f1_score:0.802293
[40]	train-f1_score:0.802168
[41]	train-f1_score:0.802249
[42]	train-f1_score:0.802035
[43]	train-f1_score:0.802068
[44]	train-f1_score:0.802237
[45]	train-f1_score:0.802321
[46]	train-f1_score:0.801893
[47]	train-f1_score:0.802016
[48]	train-f1_score:0.802014
[49]	train-f1_score:0.801766
[50]	train-f1_score:0.801596
[51]	train-f1_score:0.801937
[52]	train-f1_scor

[98]	train-f1_score:0.802238
[99]	train-f1_score:0.802324
[100]	train-f1_score:0.802531
[101]	train-f1_score:0.802533
[102]	train-f1_score:0.802447
[103]	train-f1_score:0.802535
[104]	train-f1_score:0.802663
[105]	train-f1_score:0.802748
[106]	train-f1_score:0.802789
[107]	train-f1_score:0.802705
[108]	train-f1_score:0.802706
[109]	train-f1_score:0.802539
[110]	train-f1_score:0.802497
[111]	train-f1_score:0.802581
[112]	train-f1_score:0.80258
[113]	train-f1_score:0.802918
[114]	train-f1_score:0.802961
[115]	train-f1_score:0.802877
[116]	train-f1_score:0.802706
[117]	train-f1_score:0.802916
[118]	train-f1_score:0.802831
[119]	train-f1_score:0.803001
[120]	train-f1_score:0.802959
[121]	train-f1_score:0.802958
[122]	train-f1_score:0.802832
[123]	train-f1_score:0.802958
[124]	train-f1_score:0.802999
[125]	train-f1_score:0.803126
[126]	train-f1_score:0.803085
[127]	train-f1_score:0.803128
[128]	train-f1_score:0.803003
[129]	train-f1_score:0.803128
[130]	train-f1_score:0.803253
[131]	train-f

In [ ]:
选择1

In [ ]:
迭代轮数提升

In [192]:
num_boost_round_value = [200,400,600,800,1000,1300,1600,2000,2500,3000,3500,4000]
f1_score_1_list = []

In [193]:
for i in range(len(num_boost_round_value)):
    model_1,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = 0.8,max_depth = 5,lambd = 1,subsample = 0.95,colsample_bytree = 0.95,
    min_child_weight = 1,colsample_bylevel = 0.95,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = num_boost_round_value[i],threshold_value = 0.7)
    f1_score_1_list.append(f1_score_1)   

[0]	train-f1_score:0.797637
[1]	train-f1_score:0.799413
[2]	train-f1_score:0.800494
[3]	train-f1_score:0.799812
[4]	train-f1_score:0.799438
[5]	train-f1_score:0.799625
[6]	train-f1_score:0.800298
[7]	train-f1_score:0.800253
[8]	train-f1_score:0.799809
[9]	train-f1_score:0.800021
[10]	train-f1_score:0.800928
[11]	train-f1_score:0.800929
[12]	train-f1_score:0.80088
[13]	train-f1_score:0.800635
[14]	train-f1_score:0.800716
[15]	train-f1_score:0.800545
[16]	train-f1_score:0.800463
[17]	train-f1_score:0.800177
[18]	train-f1_score:0.800591
[19]	train-f1_score:0.800678
[20]	train-f1_score:0.800851
[21]	train-f1_score:0.801177
[22]	train-f1_score:0.800723
[23]	train-f1_score:0.800887
[24]	train-f1_score:0.8008
[25]	train-f1_score:0.800552
[26]	train-f1_score:0.800891
[27]	train-f1_score:0.801056
[28]	train-f1_score:0.80142
[29]	train-f1_score:0.80134
[30]	train-f1_score:0.801709
[31]	train-f1_score:0.80171
[32]	train-f1_score:0.801918
[33]	train-f1_score:0.801999
[34]	train-f1_score:0.802127
[

[80]	train-f1_score:0.801836
[81]	train-f1_score:0.801667
[82]	train-f1_score:0.801921
[83]	train-f1_score:0.802128
[84]	train-f1_score:0.801749
[85]	train-f1_score:0.801755
[86]	train-f1_score:0.801717
[87]	train-f1_score:0.80189
[88]	train-f1_score:0.801761
[89]	train-f1_score:0.801846
[90]	train-f1_score:0.802017
[91]	train-f1_score:0.802063
[92]	train-f1_score:0.802149
[93]	train-f1_score:0.802027
[94]	train-f1_score:0.802027
[95]	train-f1_score:0.802281
[96]	train-f1_score:0.802322
[97]	train-f1_score:0.802362
[98]	train-f1_score:0.80228
[99]	train-f1_score:0.802239
[100]	train-f1_score:0.802449
[101]	train-f1_score:0.802365
[102]	train-f1_score:0.802238
[103]	train-f1_score:0.802324
[104]	train-f1_score:0.802031
[105]	train-f1_score:0.8022
[106]	train-f1_score:0.802325
[107]	train-f1_score:0.802451
[108]	train-f1_score:0.802831
[109]	train-f1_score:0.802916
[110]	train-f1_score:0.802832
[111]	train-f1_score:0.802791
[112]	train-f1_score:0.803
[113]	train-f1_score:0.803254
[114]	t

[355]	train-f1_score:0.808212
[356]	train-f1_score:0.808212
[357]	train-f1_score:0.808129
[358]	train-f1_score:0.808086
[359]	train-f1_score:0.808045
[360]	train-f1_score:0.808086
[361]	train-f1_score:0.808129
[362]	train-f1_score:0.808171
[363]	train-f1_score:0.808128
[364]	train-f1_score:0.808212
[365]	train-f1_score:0.808255
[366]	train-f1_score:0.808297
[367]	train-f1_score:0.808339
[368]	train-f1_score:0.808339
[369]	train-f1_score:0.808549
[370]	train-f1_score:0.808632
[371]	train-f1_score:0.808632
[372]	train-f1_score:0.808632
[373]	train-f1_score:0.808632
[374]	train-f1_score:0.808674
[375]	train-f1_score:0.808801
[376]	train-f1_score:0.808801
[377]	train-f1_score:0.808927
[378]	train-f1_score:0.808884
[379]	train-f1_score:0.808843
[380]	train-f1_score:0.808758
[381]	train-f1_score:0.808884
[382]	train-f1_score:0.808926
[383]	train-f1_score:0.808968
[384]	train-f1_score:0.808968
[385]	train-f1_score:0.80901
[386]	train-f1_score:0.809051
[387]	train-f1_score:0.809052
[388]	train

[232]	train-f1_score:0.805989
[233]	train-f1_score:0.805906
[234]	train-f1_score:0.805989
[235]	train-f1_score:0.805991
[236]	train-f1_score:0.805906
[237]	train-f1_score:0.805863
[238]	train-f1_score:0.805863
[239]	train-f1_score:0.805863
[240]	train-f1_score:0.805863
[241]	train-f1_score:0.805905
[242]	train-f1_score:0.805863
[243]	train-f1_score:0.805947
[244]	train-f1_score:0.806074
[245]	train-f1_score:0.806075
[246]	train-f1_score:0.805948
[247]	train-f1_score:0.805822
[248]	train-f1_score:0.805906
[249]	train-f1_score:0.805989
[250]	train-f1_score:0.806073
[251]	train-f1_score:0.806199
[252]	train-f1_score:0.806366
[253]	train-f1_score:0.806116
[254]	train-f1_score:0.806201
[255]	train-f1_score:0.806158
[256]	train-f1_score:0.806201
[257]	train-f1_score:0.8062
[258]	train-f1_score:0.806242
[259]	train-f1_score:0.806242
[260]	train-f1_score:0.806242
[261]	train-f1_score:0.806242
[262]	train-f1_score:0.806367
[263]	train-f1_score:0.80645
[264]	train-f1_score:0.806451
[265]	train-f

[507]	train-f1_score:0.813675
[508]	train-f1_score:0.813718
[509]	train-f1_score:0.813718
[510]	train-f1_score:0.813634
[511]	train-f1_score:0.813634
[512]	train-f1_score:0.813634
[513]	train-f1_score:0.81376
[514]	train-f1_score:0.813802
[515]	train-f1_score:0.813802
[516]	train-f1_score:0.813802
[517]	train-f1_score:0.813803
[518]	train-f1_score:0.813803
[519]	train-f1_score:0.813887
[520]	train-f1_score:0.813971
[521]	train-f1_score:0.813971
[522]	train-f1_score:0.814013
[523]	train-f1_score:0.814097
[524]	train-f1_score:0.814138
[525]	train-f1_score:0.814097
[526]	train-f1_score:0.814222
[527]	train-f1_score:0.814264
[528]	train-f1_score:0.814138
[529]	train-f1_score:0.814139
[530]	train-f1_score:0.814306
[531]	train-f1_score:0.814306
[532]	train-f1_score:0.814559
[533]	train-f1_score:0.814558
[534]	train-f1_score:0.814684
[535]	train-f1_score:0.814767
[536]	train-f1_score:0.814851
[537]	train-f1_score:0.814893
[538]	train-f1_score:0.814935
[539]	train-f1_score:0.814935
[540]	train

...

[2849]	train-f1_score:0.857535
[2850]	train-f1_score:0.857577
[2851]	train-f1_score:0.857619
[2852]	train-f1_score:0.857703
[2853]	train-f1_score:0.857703
[2854]	train-f1_score:0.857703
[2855]	train-f1_score:0.857661
[2856]	train-f1_score:0.857745
[2857]	train-f1_score:0.857787
[2858]	train-f1_score:0.85783
[2859]	train-f1_score:0.857913
[2860]	train-f1_score:0.857872
[2861]	train-f1_score:0.857788
[2862]	train-f1_score:0.857787
[2863]	train-f1_score:0.857829
[2864]	train-f1_score:0.857829
[2865]	train-f1_score:0.857787
[2866]	train-f1_score:0.857787
[2867]	train-f1_score:0.857913
[2868]	train-f1_score:0.857872
[2869]	train-f1_score:0.85783
[2870]	train-f1_score:0.857788
[2871]	train-f1_score:0.857788
[2872]	train-f1_score:0.857788
[2873]	train-f1_score:0.857872
[2874]	train-f1_score:0.857872
[2875]	train-f1_score:0.857914
[2876]	train-f1_score:0.857956
[2877]	train-f1_score:0.857997
[2878]	train-f1_score:0.85804
[2879]	train-f1_score:0.85804
[2880]	train-f1_score:0.858081
[2881]	train

[3115]	train-f1_score:0.862112
[3116]	train-f1_score:0.862196
[3117]	train-f1_score:0.862155
[3118]	train-f1_score:0.862155
[3119]	train-f1_score:0.862239
[3120]	train-f1_score:0.862323
[3121]	train-f1_score:0.862323
[3122]	train-f1_score:0.862406
[3123]	train-f1_score:0.862449
[3124]	train-f1_score:0.862407
[3125]	train-f1_score:0.862323
[3126]	train-f1_score:0.862365
[3127]	train-f1_score:0.862491
[3128]	train-f1_score:0.862491
[3129]	train-f1_score:0.862532
[3130]	train-f1_score:0.86249
[3131]	train-f1_score:0.862658
[3132]	train-f1_score:0.862617
[3133]	train-f1_score:0.862742
[3134]	train-f1_score:0.862868
[3135]	train-f1_score:0.86291
[3136]	train-f1_score:0.86291
[3137]	train-f1_score:0.862868
[3138]	train-f1_score:0.862868
[3139]	train-f1_score:0.862826
[3140]	train-f1_score:0.862784
[3141]	train-f1_score:0.862868
[3142]	train-f1_score:0.862953
[3143]	train-f1_score:0.862953
[3144]	train-f1_score:0.862911
[3145]	train-f1_score:0.862952
[3146]	train-f1_score:0.863078
[3147]	trai

[3381]	train-f1_score:0.866808
[3382]	train-f1_score:0.86685
[3383]	train-f1_score:0.866892
[3384]	train-f1_score:0.866934
[3385]	train-f1_score:0.866976
[3386]	train-f1_score:0.866933
[3387]	train-f1_score:0.866933
[3388]	train-f1_score:0.866933
[3389]	train-f1_score:0.866933
[3390]	train-f1_score:0.866975
[3391]	train-f1_score:0.866975
[3392]	train-f1_score:0.866975
[3393]	train-f1_score:0.867017
[3394]	train-f1_score:0.867017
[3395]	train-f1_score:0.8671
[3396]	train-f1_score:0.8671
[3397]	train-f1_score:0.8671
[3398]	train-f1_score:0.8671
[3399]	train-f1_score:0.867059
[3400]	train-f1_score:0.867059
[3401]	train-f1_score:0.867059
[3402]	train-f1_score:0.867017
[3403]	train-f1_score:0.867017
[3404]	train-f1_score:0.867059
[3405]	train-f1_score:0.867059
[3406]	train-f1_score:0.866975
[3407]	train-f1_score:0.867017
[3408]	train-f1_score:0.867059
[3409]	train-f1_score:0.867059
[3410]	train-f1_score:0.867059
[3411]	train-f1_score:0.867059
[3412]	train-f1_score:0.867101
[3413]	train-f1_s

[3647]	train-f1_score:0.87117
[3648]	train-f1_score:0.87117
[3649]	train-f1_score:0.87117
[3650]	train-f1_score:0.87117
[3651]	train-f1_score:0.87117
[3652]	train-f1_score:0.871128
[3653]	train-f1_score:0.871086
[3654]	train-f1_score:0.871128
[3655]	train-f1_score:0.87117
[3656]	train-f1_score:0.871212
[3657]	train-f1_score:0.87117
[3658]	train-f1_score:0.871212
[3659]	train-f1_score:0.871255
[3660]	train-f1_score:0.871296
[3661]	train-f1_score:0.871296
[3662]	train-f1_score:0.871255
[3663]	train-f1_score:0.87138
[3664]	train-f1_score:0.87138
[3665]	train-f1_score:0.871296
[3666]	train-f1_score:0.871422
[3667]	train-f1_score:0.871422
[3668]	train-f1_score:0.871422
[3669]	train-f1_score:0.871422
[3670]	train-f1_score:0.871422
[3671]	train-f1_score:0.871422
[3672]	train-f1_score:0.871464
[3673]	train-f1_score:0.871464
[3674]	train-f1_score:0.871506
[3675]	train-f1_score:0.871506
[3676]	train-f1_score:0.871505
[3677]	train-f1_score:0.871631
[3678]	train-f1_score:0.871548
[3679]	train-f1_s

[3913]	train-f1_score:0.875233
[3914]	train-f1_score:0.875316
[3915]	train-f1_score:0.875442
[3916]	train-f1_score:0.875442
[3917]	train-f1_score:0.8754
[3918]	train-f1_score:0.8754
[3919]	train-f1_score:0.875442
[3920]	train-f1_score:0.875358
[3921]	train-f1_score:0.875358
[3922]	train-f1_score:0.875316
[3923]	train-f1_score:0.875358
[3924]	train-f1_score:0.875484
[3925]	train-f1_score:0.875442
[3926]	train-f1_score:0.875442
[3927]	train-f1_score:0.875484
[3928]	train-f1_score:0.875484
[3929]	train-f1_score:0.875484
[3930]	train-f1_score:0.875484
[3931]	train-f1_score:0.875568
[3932]	train-f1_score:0.875568
[3933]	train-f1_score:0.87561
[3934]	train-f1_score:0.87561
[3935]	train-f1_score:0.87561
[3936]	train-f1_score:0.875652
[3937]	train-f1_score:0.875652
[3938]	train-f1_score:0.875652
[3939]	train-f1_score:0.875652
[3940]	train-f1_score:0.875652
[3941]	train-f1_score:0.875652
[3942]	train-f1_score:0.875652
[3943]	train-f1_score:0.87561
[3944]	train-f1_score:0.875652
[3945]	train-f1_

In [194]:
f1_score_1_list

[0.78768396255337392,
 0.78286888108256925,
 0.78468347880001921,
 0.78469646020105666,
 0.78594253639394163,
 0.78605929053585555,
 0.7863709285407785,
 0.78681425659592286,
 0.78671641585168517,
 0.78702748022519464,
 0.78654408872470905,
 0.78688884966210582]

In [ ]:
200反而是最好的

In [ ]:
提交部分

In [11]:
model,feature_score_1,f1_score_1,preds_1 = pipeline(
    random_seed = 1000,gamma = 0.8,max_depth = 5,lambd = 1,subsample = 0.9,colsample_bytree = 0.9,
    min_child_weight = 1,colsample_bylevel = 0.9,
    dataset1 = dataset1,dataset2 = dataset2,num_boost_round_value = 250,threshold_value = 0.65)  

[0]	train-f1_score:0.79766
[1]	train-f1_score:0.799095
[2]	train-f1_score:0.799169
[3]	train-f1_score:0.79884
[4]	train-f1_score:0.799801
[5]	train-f1_score:0.799305
[6]	train-f1_score:0.798809
[7]	train-f1_score:0.799306
[8]	train-f1_score:0.799699
[9]	train-f1_score:0.799292
[10]	train-f1_score:0.798904
[11]	train-f1_score:0.79963
[12]	train-f1_score:0.799488
[13]	train-f1_score:0.799498
[14]	train-f1_score:0.799625
[15]	train-f1_score:0.799932
[16]	train-f1_score:0.799884
[17]	train-f1_score:0.799665
[18]	train-f1_score:0.800459
[19]	train-f1_score:0.80018
[20]	train-f1_score:0.800207
[21]	train-f1_score:0.799833
[22]	train-f1_score:0.799959
[23]	train-f1_score:0.800212
[24]	train-f1_score:0.800346
[25]	train-f1_score:0.80026
[26]	train-f1_score:0.80072
[27]	train-f1_score:0.800383
[28]	train-f1_score:0.800932
[29]	train-f1_score:0.801265
[30]	train-f1_score:0.80114
[31]	train-f1_score:0.8011
[32]	train-f1_score:0.801686
[33]	train-f1_score:0.801264
[34]	train-f1_score:0.800845
[35]

In [12]:
preds = model.predict(dataset3)
preds

array([ 0.89751714,  0.86817139,  0.55805445, ...,  0.13225012,
        0.12372651,  0.12707265], dtype=float32)

In [18]:
feature_score_1

[('launch_day_near_pred', 927),
 ('launch_day_sum', 624),
 ('activity_day_near_pred', 584),
 ('device_type', 574),
 ('activity_count_sum', 487),
 ('video_count', 441),
 ('activity_page_1', 329),
 ('launch_day_sum_average', 300),
 ('activity_count_sum_average', 293),
 ('register_type', 287),
 ('activity_count_std', 226),
 ('register_day', 224),
 ('launch_day_gap_average', 211),
 ('action_type_2', 209),
 ('author_count', 206),
 ('activity_page_0', 195),
 ('activity_page_3', 180),
 ('activity_day_gap_average', 161),
 ('activity_day_sum', 159),
 ('activity_day_sum_average', 139),
 ('action_type_3', 133),
 ('video_100_count', 121),
 ('author_100_count', 114),
 ('action_type_1', 114),
 ('action_type_0', 113),
 ('activity_page_2', 73),
 ('video_day_near_pred', 64),
 ('activity_page_4', 38),
 ('author_200_count', 37),
 ('video_count_sum_average', 35),
 ('video_count_sum', 30),
 ('if_create_video', 13),
 ('video_200_count', 8),
 ('video_day_gap_average', 7),
 ('video_count_std', 6),
 ('video_da

In [17]:
preds01 = cal_preds(preds,0.8)
np.sum(preds01),len(preds01)

(24526, 51709)

In [202]:
table15_30_pred['result'] = preds01
result = table15_30_pred[table15_30_pred['result'] == 1]
result = result[['user_id']]

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
存档

In [203]:
result.to_csv('baseline_xgb_1.csv',index = False, header = False) # 基本调参后的结果

In [159]:
result.to_csv('baseline_xgb.csv',index = False, header = False)